# Fase 2: Comprensión y Exploración de Datos (El Laboratorio)

## 1\. Objetivo

Este notebook es nuestro "laboratorio" de datos. Su propósito es cargar y explorar los **4 archivos CSV** seleccionados en nuestro informe (`/docs/PHASE_2_DATA.md`) para validar que contienen las variables y llaves que necesitamos para el MVP.

No vamos a *limpiar* ni *unir* los datos aquí. Solo vamos a **inspeccionar**.

## 2\. Configuración Inicial

### 2.1. Importar Librerías

Importamos `pandas`, nuestra herramienta principal para "abrir" y "leer" los archivos CSV (nuestras "cajas" de datos).

In [1]:
import pandas as pd
import numpy as np

# Configuración para que pandas muestre más columnas si es necesario
pd.set_option('display.max_columns', 50)

### 2.2. Definir Rutas y Columnas (Nuestra "Lista de Materiales")

Como buenos ingenieros, definimos todas nuestras "constantes" en un solo lugar. Esto hace que el código sea fácil de leer y modificar.

  * `FILE_PATHS`: Las rutas relativas a nuestros 4 archivos (asumiendo que este notebook está en `/notebooks/` y los datos en `/data/raw/`).
  * `COLS_TO_LOAD`: Nuestra "Lista de Materiales" exacta (la que definimos en Fase 2). Usaremos esto para cargar *solo* las columnas que necesitamos, lo cual es una práctica de ingeniería eficiente para ahorrar memoria.

<!-- end list -->

In [4]:
# --- 1. Definir Rutas a los archivos CSV ---
# (Ajusta estas rutas si tu estructura de carpetas es diferente)
BASE_PATH = "../data/raw/"

FILE_PATHS = {
    # Bloque 1: Hogar
    "hogar_zona":   BASE_PATH + "968-Modulo1629/RECH0_2024.csv",
    "hogar_saneo":  BASE_PATH + "968-Modulo1630/RECH23_2024.csv",

    # Bloque 2: Madre
    "madre_edu":    BASE_PATH + "968-Modulo1631/REC0111_2024.csv",
    "madre_talla":  BASE_PATH + "968-Modulo1634/REC42_2024.csv",

    # Bloque 3: Niño
    "niño_peso":    BASE_PATH + "968-Modulo1633/REC41_2024.csv",
    "niño_target":  BASE_PATH + "968-Modulo1638/REC44_2024.csv"
}

# --- 2. Definir nuestra "Lista de Materiales" (Bill of Materials) ---
# Estas son las únicas columnas que cargaremos en memoria.

COLS_TO_LOAD = {
    # Bloque 1
    "hogar_zona": [
        "HHID",    # Llave de Hogar
        "HV025",   # Predictor: Zona U/R
    ],
    "hogar_saneo": [
        "HHID",    # Llave de Hogar
        "HV201",   # Predictor: Agua
        "HV205",   # Predictor: Saneamiento
        "HV270"    # Predictor: Riqueza
    ],

    # Bloque 2
    "madre_edu": [
        "CASEID",  # Llave de Cuestionario (Madre)
        "HHID",    # ¡Llave Puente a Hogar!
        "V106",    # Predictor: Educación Madre
    ],
    "madre_talla": [
        "CASEID",  # Llave de Cuestionario (Madre)
        "V438"     # Predictor: Talla Madre
    ],

    # Bloque 3
    "niño_peso": [
        "CASEID",  # Llave de Cuestionario (Madre)
        "MIDX",    # Llave de Niño (Índice)
        "M19"      # Predictor: Peso Nacer (kilos)
    ],
    "niño_target": [
        "CASEID",  # Llave de Cuestionario (Madre)
        "HWIDX",   # Llave de Niño (Índice)
        "HW1",     # Predictor: Edad Niño (meses)
        "HW70"     # OBJETIVO: Talla/Edad (HAZ)
    ]
}

## 3. Exploración por Bloque
Inspeccionaremos cada bloque de nuestro diagrama de flujo.

### 3.1. Bloque 1: HOGAR (Llave HHID)
Cargamos y validamos los dos archivos que definen el hogar.

#### 3.1.1. Archivo RECH0.csv (Zona)

In [ ]:
# --- Carga ---
# Forzamos la llave HHID a ser texto (string) para evitar problemas en la unión.
try:
    df_hogar_zona = pd.read_csv(
        FILE_PATHS["hogar_zona"],
        usecols=COLS_TO_LOAD["hogar_zona"],
        dtype={'HHID': str}
    )

    print(f"Éxito: Se cargó df_hogar_zona con {df_hogar_zona.shape[0]} filas y {df_hogar_zona.shape[1]} columnas.")

    # --- Inspección ---
    print("\n--- .head() ---")
    print(df_hogar_zona.head())

    # .info() es la radiografía del ingeniero: nos dice tipos de datos y nulos.
    print("\n--- .info() ---")
    df_hogar_zona.info()

    # Inspección Específica
    print("\n--- Valores en 'HV025' (Zona U/R) ---")
    print(df_hogar_zona['HV025'].value_counts(dropna=False))

except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo en {FILE_PATHS['hogar_zona']}")
except Exception as e:
    print(f"Un error ocurrió al cargar RECH0: {e}")

Éxito: Se cargó df_hogar_zona con 37390 filas y 2 columnas.

--- .head() ---
              HHID  HV025
0        325502001      1
1        325503101      1
2        325503901      1
3        325504001      1
4        325504701      1

--- .info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37390 entries, 0 to 37389
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   HHID    37390 non-null  object
 1   HV025   37390 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 584.3+ KB

--- Valores en 'HV025' (Zona U/R) ---
HV025
1    24695
2    12695
Name: count, dtype: int64


#### 3.1.2. Archivo RECH23.csv (Saneamiento y Riqueza)

In [ ]:
# --- Carga ---
try:
    df_hogar_saneo = pd.read_csv(
        FILE_PATHS["hogar_saneo"],
        usecols=COLS_TO_LOAD["hogar_saneo"],
        dtype={'HHID': str}
    )

    print(f"Éxito: Se cargó df_hogar_saneo con {df_hogar_saneo.shape[0]} filas y {df_hogar_saneo.shape[1]} columnas.")

    # --- Inspección ---
    print("\n--- .head() ---")
    print(df_hogar_saneo.head())

    print("\n--- .info() ---")
    df_hogar_saneo.info()

    # Inspección Específica
    print("\n--- Valores en 'HV201' (Agua) ---")
    print(df_hogar_saneo['HV201'].value_counts(dropna=False).head(10))

    print("\n--- Valores en 'HV205' (Saneamiento) ---")
    print(df_hogar_saneo['HV205'].value_counts(dropna=False).head(10))

    print("\n--- Valores en 'HV270' (Riqueza) ---")
    print(df_hogar_saneo['HV270'].value_counts(dropna=False))

except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo en {FILE_PATHS['hogar_saneo']}")
except Exception as e:
    print(f"Un error ocurrió al cargar RECH23: {e}")

Éxito: Se cargó df_hogar_saneo con 37390 filas y 4 columnas.

--- .head() ---
              HHID HV201 HV205 HV270
0        325502001                  
1        325503101    11    22     2
2        325503901    11    11     3
3        325504001    11    22     1
4        325504701    71    11     2

--- .info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37390 entries, 0 to 37389
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   HHID    37390 non-null  object
 1   HV201   37390 non-null  object
 2   HV205   37390 non-null  object
 3   HV270   37390 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB

--- Valores en 'HV201' (Agua) ---
HV201
11    22486
71     5753
       3372
12     1454
96      927
61      713
43      690
41      605
13      571
21      502
Name: count, dtype: int64

--- Valores en 'HV205' (Saneamiento) ---
HV205
11    19997
23     5403
       3372
21     2556
22     2401
32     2128
12     1433


<b>Validación Exitosa:</b> 
- Los dos archivos (RECH0 y RECH23) se cargaron correctamente. Ambos contienen la llave HHID (tipo object) con 37,390 filas no nulas, lo que valida nuestra estrategia de unión (join) para este bloque.

<b>Hallazgo (RECH0):</b> 
- El archivo RECH0 está "limpio" de fábrica. El predictor HV025 (Zona) se cargó correctamente como int64 (numérico) y sus value_counts (1 y 2) coinciden con lo esperado (Urbano/Rural).

<b>Riesgo Identificado (RECH23):</b> 
- El archivo RECH23 presenta un problema de calidad de datos. Las columnas HV201 (Agua), HV205 (Saneo) y HV270 (Riqueza) se cargaron como object (texto), pero sus valores son números.

<b>Diagnóstico:</b> 
- La salida de .info() (Dtype: object) y .value_counts() (que muestra que el valor más común, 3372 veces, es una cadena vacía ) confirma que estas columnas están contaminadas con cadenas de texto vacías (' ').

<b>Acción (Para Fase 3):</b>

- Limpieza (RECH23): Estas tres columnas (HV201, HV205, HV270) requerirán una limpieza robusta en dos pasos: primero, quitar todos los espacios en blanco (.strip()), y luego   transformar los vacíos en Nulos/NaN.

<b>Ingeniería de Características (RECH0):</b>
- Todas las variables predictoras de este bloque (HV025, HV201, HV205, HV270) son categóricas. Para la Fase 3, documentamos un plan de mapeo (mapping) para traducirlas a palabras (ej. HV025: {1: 'Urbano'}, HV270: {1: 'Más Pobre', ...}) para el análisis EDA (gráficos). Posteriormente, las prepararemos para la codificación (encoding) adecuada (ej. One-Hot Encoding) que requieren los modelos de ML.

### 3.2. Bloque 2: MADRE (Llave CASEID)
Cargamos los archivos de la madre y validamos la llave puente (HHID).

#### 3.2.1. Archivo REC0111.csv (Educación y Llave Puente)

In [ ]:
# --- Carga ---
# Forzamos AMBAS llaves (CASEID y HHID) a ser texto.
try:
    df_madre_edu = pd.read_csv(
        FILE_PATHS["madre_edu"],
        usecols=COLS_TO_LOAD["madre_edu"],
        dtype={'CASEID': str, 'HHID': str}
    )

    print(f"Éxito: Se cargó df_madre_edu con {df_madre_edu.shape[0]} filas y {df_madre_edu.shape[1]} columnas.")

    # --- Inspección ---
    print("\n--- .head() ---")
    print(df_madre_edu.head())

    print("\n--- .info() ---")
    # ¡Verificar que HHID (la llave puente) no tenga nulos!
    df_madre_edu.info()

    # Inspección Específica
    print("\n--- Valores en 'V106' (Educación) ---")
    print(df_madre_edu['V106'].value_counts(dropna=False))

except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo en {FILE_PATHS['madre_edu']}")
except Exception as e:
    print(f"Un error ocurrió al cargar REC0111: {e}")

Éxito: Se cargó df_madre_edu con 37117 filas y 3 columnas.

--- .head() ---
               CASEID             HHID V106
0        325503101  2        325503101    2
1        325503101  4        325503101    2
2        325503901  2        325503901    2
3        325504701  2        325504701    2
4        325505001  1        325505001    1

--- .info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37117 entries, 0 to 37116
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CASEID  37117 non-null  object
 1   HHID    37117 non-null  object
 2   V106    37117 non-null  object
dtypes: object(3)
memory usage: 870.1+ KB

--- Valores en 'V106' (Educación) ---
V106
2    17124
3    10460
1     6326
      2865
0      342
Name: count, dtype: int64


#### 3.2.2. Archivo REC42.csv (Talla de la Madre)

In [ ]:
# --- Carga ---
try:
    df_madre_talla = pd.read_csv(
        FILE_PATHS["madre_talla"],
        usecols=COLS_TO_LOAD["madre_talla"],
        dtype={'CASEID': str}
    )

    print(f"Éxito: Se cargó df_madre_talla con {df_madre_talla.shape[0]} filas y {df_madre_talla.shape[1]} columnas.")

    # --- Inspección ---
    print("\n--- .head() ---")
    print(df_madre_talla.head())

    print("\n--- .info() ---")
    df_madre_talla.info()

    # Inspección Específica
    print("\n--- Descripción de 'V438' (Talla Madre) ---")
    # .describe() nos da estadísticas. Buscamos valores extraños.
    # El diccionario (REC42.pdf, pág 3) dice que la talla está en cm (1d)
    # y '9998' = Casos marcados.
    print(df_madre_talla['V438'].describe())

    print("\nValores atípicos ( > 9000) en 'V438':")
    print(df_madre_talla[df_madre_talla['V438'] > 9000]['V438'].value_counts())

except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo en {FILE_PATHS['madre_talla']}")
except Exception as e:
    print(f"Un error ocurrió al cargar REC42: {e}")

Éxito: Se cargó df_madre_talla con 34252 filas y 2 columnas.

--- .head() ---
               CASEID  V438
0        325503101  2  1525
1        325503101  4  1518
2        325503901  2  1602
3        325504701  2  1443
4        325505001  1  1486

--- .info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34252 entries, 0 to 34251
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CASEID  34252 non-null  object
 1   V438    34252 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 535.3+ KB

--- Descripción de 'V438' (Talla Madre) ---
count    34252.000000
mean      1645.156925
std        998.593461
min       1170.000000
25%       1488.000000
50%       1526.000000
75%       1565.000000
max       9999.000000
Name: V438, dtype: float64

Valores atípicos ( > 9000) en 'V438':
V438
9999    481
Name: count, dtype: int64


### Veredicto (Bloque 2: Madre)

  * <b>Validación Exitosa (¡Crítico\!):</b> ¡Nuestro "archivo puente" está 100% validado\! El archivo `REC0111` (`df_madre_edu`) contiene tanto la llave `CASEID` como la llave `HHID`. Esto confirma nuestra arquitectura de unión: podemos unir a la madre con el niño (usando `CASEID`) y con el hogar (usando `HHID`).

  * <b>Riesgo Identificado (Tipo 1: Texto Contaminado):</b> La columna `V106` (Educación) en `REC0111` se cargó como `object` (texto). La salida de `.info()` y `.value_counts()` (que muestra 2865 cadenas vacías `     ` y valores como `" 2"`) confirma que, al igual que el Bloque 1, está **contaminada con espacios y cadenas vacías**.

  * <b>Riesgo Identificado (Tipo 2: Código de Error Numérico):</b> La columna `V438` (Talla Madre) en `REC42` se cargó *correctamente* como `int64` (numérico). Sin embargo, nuestro filtro de "Valores atípicos" detectó 481 instancias del código de error `9999` ("Casos marcados" / "Missing").

  * <b>Acción (Plan para Fase 3):</b>

    1.  **Limpieza (`V106`):** Requerirá la limpieza de dos pasos: primero, **quitar todos los espacios en blanco (`.strip()`)**, y luego **convertir a tipo string** (lo que transformará los vacíos en Nulos/NaN) para un posterior "encoding".
    2.  **Limpieza (`V438`):** Requerirá una limpieza diferente: **reemplazar el valor numérico** `9999` por un Nulo estándar (`np.nan`).
    3.  <b>Ingeniería de Características (`V106`):</b> Esta variable es categórica. En Fase 3, la **mapearemos** a sus etiquetas de texto (ej. `{0: 'Sin Educación', 1: 'Primaria', 2: 'Secundaria', 3: 'Superior'}`) para el análisis EDA y luego la prepararemos para la **codificación (encoding)** del modelo.
    4.  <b>Transformación (`V438`):</b> El diccionario `REC42.pdf` indica que `V438` es "Talla... (cms-1d)", lo que significa que un valor como `1525` es `152.5 cm`. Para nuestro modelo, **dividiremos esta columna por 10** en la Fase 3 para estandarizarla a centímetros (ej. `152.5`).

### 3.3. Bloque 3: NIÑO (Llave CASEID + Índice MIDX/HWIDX)
Este es el bloque más importante. Contiene nuestro OBJETIVO.

#### 3.3.1. Archivo REC41.csv (Peso al Nacer)

In [ ]:
# --- Carga ---
# Forzamos llaves a ser texto
try:
    df_niño_peso = pd.read_csv(
        FILE_PATHS["niño_peso"],
        usecols=COLS_TO_LOAD["niño_peso"],
        dtype={'CASEID': str, 'MIDX': str}
    )

    print(f"Éxito: Se cargó df_niño_peso con {df_niño_peso.shape[0]} filas y {df_niño_peso.shape[1]} columnas.")

    # --- Inspección ---
    print("\n--- .head() ---")
    print(df_niño_peso.head())

    print("\n--- .info() ---")
    df_niño_peso.info()

    # Inspección Específica
    print("\n--- Descripción de 'M19' (Peso Nacer) ---")
    # El diccionario (REC41.pdf, pág 2) dice que M19 es 'Peso... (kilos-3 dec.)'
    # Y usa '9998' = No sabe, '9996' = No se pesó.
    print(df_niño_peso['M19'].describe())

    print("\nValores atípicos ( > 90) en 'M19' (deberían ser kilos):")
    # Si M19 está en kilos, no debería ser mayor a, digamos, 10.
    # Pero el diccionario dice 3 decimales, así que '9998' podría no ser el único código.
    # Ah, espera, M19 en el diccionario es 'N', 4 de longitud.
    # Vamos a asumir que los códigos > 90 son "missing".
    print(df_niño_peso[df_niño_peso['M19'] > 90]['M19'].value_counts())

except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo en {FILE_PATHS['niño_peso']}")
except Exception as e:
    print(f"Un error ocurrió al cargar REC41: {e}")


Éxito: Se cargó df_niño_peso con 19751 filas y 3 columnas.

--- .head() ---
               CASEID MIDX   M19
0        325503101  2    1  2550
1        325504701  2    1  3195
2        325505001  1    1  2970
3        325508901  2    1  2700
4        325509701  2    1  3420

--- .info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19751 entries, 0 to 19750
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CASEID  19751 non-null  object
 1   MIDX    19751 non-null  object
 2   M19     19751 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 463.0+ KB

--- Descripción de 'M19' (Peso Nacer) ---
count    19751.000000
mean      3446.104349
std       1289.733526
min        360.000000
25%       2925.000000
50%       3280.000000
75%       3620.000000
max       9998.000000
Name: M19, dtype: float64

Valores atípicos ( > 90) en 'M19' (deberían ser kilos):
M19
3500    788
3200    748
3000    714
3800    668
2800    666
   

#### 3.3.2. Archivo REC44.csv (¡Nuestro OBJETIVO!)

In [ ]:
# --- 3.3.2. Inspección de Variables (Antropometría) ---
# CORREGIDO para manejar columnas de texto ('object')

if 'df_niño_target' in locals():
    # --- HW1 (Edad Niño en meses) ---
    print("\n--- Descripción de 'HW1' (como texto) ---")
    # .describe() en una columna 'object' nos mostrará 'unique', 'top' (el más común), 'freq'
    print(df_niño_target['HW1'].describe())

    print("\n--- Primeros 20 valores más comunes en 'HW1' (como texto) ---")
    # .value_counts() es seguro en columnas de texto y nos revelará
    # si hay espacios en blanco (ej. " 18") o códigos de error.
    print(df_niño_target['HW1'].value_counts().head(20))


    # --- HW70 (OBJETIVO: Talla/Edad - HAZ) ---
    print("\n--- Descripción de 'HW70' (como texto) ---")
    print(df_niño_target['HW70'].describe())

    print("\n--- Primeros 20 valores más comunes en 'HW70' (como texto) ---")
    # Esta es la línea que reemplaza al filtro numérico que falló.
    # Esto nos mostrará los '9998' (o valores similares) como texto.
    print(df_niño_target['HW70'].value_counts().head(20))


--- Descripción de 'HW1' (como texto) ---
count     19751
unique       61
top            
freq        517
Name: HW1, dtype: object

--- Primeros 20 valores más comunes en 'HW1' (como texto) ---
HW1
      517
13    386
31    369
9     359
51    358
10    357
48    354
7     353
12    352
11    351
28    350
40    348
43    347
49    346
23    344
20    344
33    342
29    341
56    341
38    339
Name: count, dtype: int64

--- Descripción de 'HW70' (como texto) ---
count     19751
unique      666
top            
freq        517
Name: HW70, dtype: object

--- Primeros 20 valores más comunes en 'HW70' (como texto) ---
HW70
        517
9999    266
-69     103
-103     92
-87      91
-78      88
-96      85
-79      85
-126     85
-63      84
-110     84
-42      83
-85      83
-90      82
-94      81
-48      81
-109     80
-70      80
-116     79
-127     79
Name: count, dtype: int64


### Veredicto (Bloque 3: Niño)

  * <b>Validación Exitosa (¡Crítico\!):</b> ¡Nuestra granularidad de unión está 100% validada\! Los archivos `REC41` y `REC44` se cargaron y ambos contienen la llave `CASEID` para unir con la madre.

  * Lo más importante: `REC41` tiene la llave `MIDX` y `REC44` tiene la llave `HWIDX`. Esto **confirma nuestra estrategia de unión (join)** del bloque Niño, donde `MIDX` y `HWIDX` se tratarán como la misma llave (`LLAVE_NINIO`).

  * <b>Riesgo Identificado (Tipo 1: Código de Error Numérico):</b>

      * La columna `M19` (Peso Nacer) se cargó correctamente como `int64` (numérico).
      * Sin embargo, el diccionario `REC41.pdf` nos advierte de códigos de error (ej. `9996`="No se pesó", `9998`="No sabe"). Aunque no son visibles en el `.describe()` truncado, debemos tratarlos.

  * <b>Riesgo Identificado (Tipo 2: Texto Contaminado):</b>

      * Las columnas `HW1` (Edad Niño) y `HW70` (Target HAZ) se cargaron incorrectamente como `object` (texto).
      * **Diagnóstico:** La salida de `value_counts()` (ej. `top: ' '` con 517 instancias, y valores como `" 13"`) **confirma que la causa** es una contaminación por **cadenas vacías (`' '`)** y **espacios en blanco iniciales** (ej. `" 13"`, `" -70"`).

  * <b>Acción (Plan para Fase 3):</b>

    1.  **Limpieza (`M19`):** Requerirá **reemplazar los códigos numéricos** de error (ej. `[9996, 9998]`) por un Nulo estándar (`np.nan`).
    2.  **Limpieza (`HW1`, `HW70`):** Requerirán la limpieza de dos pasos: primero, **quitar todos los espacios en blanco (`.strip()`)**, y luego **convertir a tipo numérico** (`pd.to_numeric(errors='coerce')`), lo que transformará los vacíos en Nulos/NaN.
    3.  **Transformación (`M19`):** La data de `M19` (ej. `mean=3446`) está claramente en **gramos**. Para estandarizarla y mejorar la interpretabilidad del modelo, la **dividiremos por 1000** en la Fase 3 para convertirla a **kilos** (ej. `3.446 kg`).
